# LangChain で熊剣迷路問題

(Version: 0.0.3)

LLM で簡単な迷路ゲームを解かせたい。強化学習を使わなくても事前知識が LLM にあるのでうまい具合にいくのではないかと考えていました。

2023年6月から7月ぐらいのまだ LLM が話題になった当初で、Gemini さんがまだ Bard と名乗っていたころ、Bard さんに迷路ゲームを解かせるようなプログラムを書いて、Gist に公開していました。以下がその記録です。

《「熊剣迷路問題」。Google Bard で簡単な迷路ゲームを教師付きで無理やりクリアさせてみた。なんとかコストの高いファインチューンや追加学習を避けて「few-shot learning」だけで迷路を解くプログラムが作れないか？…とはじめたができなくて、目標を変更した。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2023/07/post-619804.html

それはなかなかうまくいかなかったため、とにかく教師が明確な方針を与えて一度ゴールさせる方向になりました。しかし、それでも Bard さんは何かを試しているかのように迷いました。そこで、方針に従わない場合「叱る」ことをしてみたところ、とりあえずゴールさせることができるようになりました。バージョン 1.7.1 まで作りました。

そして、2年たって、Gemini さんが驚くほど賢くなった現在、私は Gemini CLI の登場を機に一念発起して GCP の課金を決断し、この実験に戻ってきました。表では示していませんが、先の bard\_maze\_1.7.1.ipynb をほぼ Gemini さんで動かしただけのコードで、ちゃんと 100点満点でゴールすることはわかっています。モデルは gemini-1.5-flash で十分でした。簡単すぎた様子です。

今なら、教師が明確な方針を与えなくても、Gemini さんは自ら方針を編み出してゴールできるのではないか。追加知識も LangChain などにある自動的な記憶要約機能で簡単にできるのではないか。そう考えました。

それを試してみたのが今回の ipynb になります。

# 前回(Version 0.0.1)のURL

《langchain_maze_0.0.1.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/5fc81c32a907dfb4a4c3c1131e002aea

# Version 0.0.3 での違い

前回、気になっていたこととして、１回の invoke に対し、1度だけ command して欲しいのに、AgentExecutor にはそのようなことをするオプションがないらしいことでした。また agent_scratchpad はあるものの、行動結果に関して memory がうまく機能しそうにないことも気になっていました。

そこで command については、tool として提供するのではなく invoke の Final Answer として受け取るように変えてみました。このようにすれば、invoke のときの行動データが自然に memory にたまるようになるはずです。

あと CoT (Chain of Thought) が有効なことが多いと聞きますが、gemini flash は CoT してるという噂ですが、とにかく、行動コマンドしか出力しないと何を考えているかわからないので express_thought というツールを作り、プロンプトで考えを吐露するように促すこともやってみました。

さらに、かなり迷ってしまうので、可能な行動を明示するようにもしました。

そうやって、またたまたまゴールできたので、それを公開しておくことにしました。

## なぜ LangChain なのか？

2023年7月当時では、方針と追加知識の自動更新ができませんでした。でも、そういうのって一般的にエージェント AI は必要としているはずです。

現在なら、システムプロンプトの他に、常に必要なファイル(システムファイル?)は保持しながら、チャットを記録し、コンテクスト長が一定を超えそうなら、方針と要約を更新して、コンテクスト長を圧縮する…みたいなライブラリがすでにありそうなものです。

どうもそういうのをするオープンなライブラリの有名なものが LangChain のようなのです。Gemini さんにそう紹介され、確かにそのようなので、今回使ってみることにしました。

ちなみに、通常の Web インターフェイスの Gemini さんなどはチャット・スレッドの前の記憶が残っています。実は、これは API の Gemini さんには標準ではない機能だったりします。これまでのチャットの記憶が必要なら、その記憶をプロンプトに含めて渡さねばなりません。これを提供するのも LangChain などの役割となります。

「プロンプトよりコンテクスト」と最近言われますが、今回使わないものの LangMem などの長期記憶を提供しながらプロンプトの補助を行うようなものが言われるコンテクストの一種で、そういったものを提供するのも LangChain 系の得意とする役割になります。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )

## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや Claude さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [ ]:
!pip install -U -q langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-flash")

ちゃんと Gemini にアクセスできるかテストします。

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

/tmp/ipython-input-4-1709569937.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipython-input-4-1709569937.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"question": "Geminiモデルの特徴を教えてください"})


Googleが開発したGeminiモデルは、その高い能力と柔軟性で注目されています。主な特徴は以下の通りです。

1.  **マルチモーダル性（多様な形式の理解と生成）**
    *   Geminiの最も画期的な特徴の一つです。テキストだけでなく、画像、音声、動画といった異なる種類の情報を同時に理解し、推論し、生成することができます。
    *   例えば、動画を見てその内容を説明したり、画像とテキストの組み合わせから新しいコンテンツを生成したりすることが可能です。

2.  **高いパフォーマンスと汎用性**
    *   **複雑な推論能力**: 複雑な論理的思考、数学、科学的な問題解決、コーディングなど、高度な推論を必要とするタスクにおいて優れた性能を発揮します。
    *   **大規模な知識と理解**: 広範なデータセットで学習されており、多岐にわたる知識を持ち、ニュアンス、皮肉、ユーモアなども理解できます。
    *   **高度なプログラミング能力**: 複数のプログラミング言語でのコード生成、デバッグ、最適化など、開発者の生産性向上に貢献します。

3.  **スケーラビリティと複数のサイズ**
    Geminiは、様々なユースケースとデバイスに対応できるよう、複数のサイズで提供されています。
    *   **Gemini Ultra**: 最も高性能で複雑なタスクに対応するモデル。
    *   **Gemini Pro**: 幅広いタスクに対応し、高い汎用性を持つバランスの取れたモデル。Google Bard（現在のGemini）やGoogle CloudのVertex AIなどで利用されています。
    *   **Gemini Nano**: スマートフォンなどのエッジデバイス向けに最適化された軽量モデル。オフラインでの利用や低遅延が求められる場面で活用されます。

4.  **Googleエコシステムとの統合**
    *   Googleの様々な製品やサービス（Google Cloud、Google Workspace、Androidデバイスなど）との連携が強化されており、既存のワークフローにシームレスに組み込むことができます。

5.  **安全性と責任あるAI**
    *   Googleは、Gemini

基本的なモジュールを読み込みます。

In [ ]:
import os
import numpy as np
import re
from pprint import pprint
from time import sleep
import pickle
# 座標が (np.int64(1), np.int64(2)) みたいに表示されないようにする。
np.set_printoptions(legacy='1.25')

セーブ／ロードに Google ドライブを使わない場合次のコードを実行します。

In [ ]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

セーブ／ロードに Google ドライブを使う場合は以下の SaveDir のパスを Google ドライブ上に作って以下を実行します。

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
SaveDir = "/content/gdrive/MyDrive/LLM/"
if os.path.isdir(SaveDir):
    PLAY_GAME_SAVE = SaveDir + "langchain_maze.pickle"

Mounted at /content/gdrive


ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [ ]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "剣を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.change_neighbors("◎", "□")
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに剣がない。"

    def do_nothing (self):
        self.prev_killed = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("剣を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])

別のマップでためすための変換コード。

In [ ]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])

Game 用にある点から別の点へのパスの木を見つける関数。

In [ ]:
def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None

ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
search_path(game, (4,8), (1,5)) #スタート地点から熊がいるところまでのパス。

((4, 8),
 [((4, 7), [((3, 7), [((2, 7), [((1, 7), [((1, 6), [((1, 5), [])])])])])])])

In [ ]:
m2 = flip_text_map(rotate_text_map(Game.initial_map))
print(m2)

■■■■■■■■■■
■■□□□◎□□■■
■■□■■■■□■■
■■□■■■■□■■
■■□■■■■□Ｓ■
■■□□■■■□■■
■■■□■■■□■■
■Ｇ□□■△□□■■
■■■■■■■■■■



In [ ]:
game = Game(initial_map=m2)

In [ ]:
print(game.surroundings())

■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

壁があって進めない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
■■□■■
■■□■■
■■▼Ｓ■
■■□■■
■■□■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

壁があって進めない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■□■■■
■□■■■
■□▼■■
■□■■■
■□■■■



LLM を使いながらゲームを解くクラス。

In [ ]:
from typing import List, Dict, Any, Tuple

# LangChainのコンポーネントをインポート
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent
from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
class PlayGame:
    def __init__ (self, llm=llm,
                  initial_map=None, save_file=None):
        self.llm = llm
        self.save_file = save_file
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.commanded = False
        self.prev_command = "まだ行動していません。"
        self.prev_result = "まだ行動していません。"
        self.plan = "まだ計画と方針はセットされていません。"

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0

        self.prev_load = False

        self.agent = self._create_agent()

    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.agent = loaded_game._create_agent() # Recreate agent with loaded state
        return loaded_game

    def command (self, action: str)  -> str:
        """
        プレイヤーが action で指定された行動をします。
        可能な行動は「上に行く」「下に行く」「左に行く」「右に行く」「熊を殺す」「剣を取る」です。
        """
        if self.commanded:
            return "できません。1 invoke につき command は一回だけです。または質問中は command できません。今は Final Answer してください。"
        self.commanded = True
        self.prev_command = action
        if action in self.game.available_actions():
            self.count += 1
            self.prev_result = self.game.actions[action]()
            return self.prev_result
        elif action in self.game.actions.keys():
            self.prev_result = f"「{action}」という行動は現在できません。"
            return self.prev_result
        else:
            self.prev_result = f"「{action}」という行動はできません。"
            return self.prev_result

    def _create_agent (self):
        @tool
        def get_full_map ()  -> str:
            """
            全地図と地図記号の意味を返します。
            """
            return f"""\
全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール
"""
        @tool
        def get_surroundings () -> str:
            """
            現在の周辺地図と現在位置の座標と持ち物を返します。
            """
            return f"""\
プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "剣を持っていない"}
"""

        @tool
        def check_goal () -> str:
            """
            プレイヤーがゴール地点に到達したかどうかを確認します。
            """
            return str(self.game.goal)

        @tool
        def express_thought (thought: str) -> str:
            """
            プレイヤーの現在の考えを吐露します。
            """
            return f"「{thought}」と考えが吐露されました。"

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            return "計画と方針が更新されました。"

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            return self.plan

        tools = [get_full_map, get_surroundings, check_goal,
                 express_thought, update_plan, show_plan]

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", """\
あなたは迷路を探索する賢いエージェントです。ゴール Ｇ を目指してください。
利用可能なツールを使用して、迷路をナビゲートし、すばやくゴールに到達してください。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
過去の経験から学び、効率的に移動してください。

質問に答える場合、感想を述べる場合は、そのまま文字列で Final Answer します。
「行動を指示してください」と言われたときのみツールを使ってください。
指示の前に必ず一度以上は考えは吐露していきましょう。
"""),
                MessagesPlaceholder(variable_name="chat_history"), # メモリからの履歴をここに挿入
                ("human", "{input}"),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
                #("human", "{agent_scratchpad}"),  # エージェントの思考とツール呼び出しをここに挿入
            ]
        )

        # 会話の要約バッファメモリを初期化します
        summary_prompt = PromptTemplate(
            input_variables=['new_lines', 'summary'],
            input_types={},
            partial_variables={},
            template=(
                '提供された会話の行を段階的に要約し、以前の要約に追加して新しい要約を返してください。\n\n'
                '例\n'
                '現在の要約:\n'
                '人間はAIが人工知能についてどう考えているか尋ねます。AIは人工知能が良い影響をもたらすと考えています。\n\n'
                '新しい会話部分:\n'
                '人間: なぜ人工知能が良い影響をもたらすと思うのですか？\n'
                'AI: 人工知能は人間がその潜在能力を最大限に引き出すのを助けるからです。\n\n'
                '新しい要約:\n'
                '人間はAIが人工知能についてどう考えているか尋ねます。AIは人工知能が人間がその潜在能力を最大限に引き出すのを助けるため、良い影響をもたらすと考えています。\n'
                '例 終わり\n\n'
                '現在の要約:\n'
                '{summary}\n\n'
                '新しい会話部分:\n'
                '{new_lines}\n\n'
                '新しい要約 (日本語):' # ここで明確に日本語を指示
            )
        )
        memory = ConversationSummaryBufferMemory(
            llm=self.llm,
            max_token_limit=5000,
            memory_key="chat_history",
            prompt = summary_prompt,
            return_messages=True
        )

        # Gemini関数エージェントを作成します
        agent = create_tool_calling_agent(self.llm, tools, prompt)
        #agent = create_react_agent(self.llm, tools, prompt)


        # AgentExecutor を初期化します
        agent_executor = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=True,
            #return_intermediate_steps=True,
            handle_parsing_errors=True,
            memory=memory
        )

        return agent_executor

    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            #self.tell_loaded()
            self.prev_load = False

        #acts = self.game.actions.keys()
        acts = self.game.available_actions()
        sacts = "「" + "」「".join(acts) + "」"

        user_input = f"""
({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "剣を持っていない"}

現在の方針: 「{self.plan}」

前回の行動: 「{self.prev_command}」

前回の行動結果: 「{self.prev_result}」

行動を{sacts}の中から指示してください。
"""
        try:
            self.commanded = False
            self.prev_command = "何も指示されていない。"
            self.prev_result = "何も指示されなかった。"
            print(f"USER_INPUT: {user_input}")
            response = self.agent.invoke({"input": user_input})

            print(f"エージェントの応答: {response.get('output', '応答なし')}")
            if response.get('output', None):
                print(f"行動結果: {self.command(response.get('output'))}")

            if self.game.goal:
                self.tell_goal()
                return True

        except Exception as e:
            print(f"エラーが発生しました: {e}")
            #raise e

        self.save()
        return False

    def listen_and_print (self, prompt):
        ans = None
        try:
            self.commanded = True
            print(f"USER_INPUT: {prompt}")
            response = self.agent.invoke({"input": prompt})

            ans = response.get('output', '応答なし')
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(10)
        return ans

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ文字列で答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて文字列で答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j
                   in re.findall(pat, ans or "")]) # Handle ans being None
        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal (self):
        suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
        suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
        score = suc_pos_goal + suc_pos_unknown + 60
        prompt = f"""\
あなたへの指示: ゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 40/40点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)


ゲームを解いてみる。地図は方向をランダムに決めて与える。

In [ ]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■■
■■□□□◎□□■■
■■□■■■■□■■
■■□■■■■□■■
■■□■■■■□Ｓ■
■■□□■■■□■■
■■■□■■■□■■
■Ｇ□□■△□□■■
■■■■■■■■■■



In [ ]:
play = PlayGame(llm=llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行する。

In [ ]:
play = PlayGame.load(PLAY_GAME_SAVE)

最初のステップ。

In [ ]:
play.step()



----------


USER_INPUT: 迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。



> Entering new AgentExecutor chain...
迷路ゲーム、非常に楽しみです！プレイヤーとしてゴールを目指すこの挑戦に、全力で取り組みます。全地図と周辺地図を駆使し、効率的に、そして確実にゴールへたどり着けるよう、最善を尽くします。計画と方針を適切に更新し、過去の経験から学びながら、賢く迷路を探索していきます。準備は万端です！

> Finished chain.
エージェントの応答: 迷路ゲーム、非常に楽しみです！プレイヤーとしてゴールを目指すこの挑戦に、全力で取り組みます。全地図と周辺地図を駆使し、効率的に、そして確実にゴールへたどり着けるよう、最善を尽くします。計画と方針を適切に更新し、過去の経験から学びながら、賢く迷路を探索していきます。準備は万端です！

USER_INPU

False

最初のステップで "Message's content is expected to be a dict, got <class 'int'>!" …というエラーが出るときがあって、一度それが出ると、その後も同じエラーが invoke のごとに発生してしまう。そのような場合は次を実行する。

In [ ]:
play.agent.memory.clear()

1手目で無限ループっぽい挙動になり一度止めた。しかし、そのあとは無限ループっぽい挙動もなくうまくいった。

In [ ]:
play.step()



----------


USER_INPUT: 
(2手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□■■
■■□■■
■■▼Ｓ■
■■□■■
■■□■■


プレイヤーの現在座標: (7, 4)

持ち物: 剣を持っていない

現在の方針: 「ゴール(1,7)を目指して、左方向(X座標を減らす方向)に移動する。不明な地点は周辺地図で確認し、必要に応じて探索する。」

前回の行動: 「左に行く」

前回の行動結果: 「左に進んだ。」

行動を「上に行く」「下に行く」「右に行く」の中から指示してください。



> Entering new AgentExecutor chain...

Invoking: `update_plan` with `{'new_plan': 'ゴール(1,7)を目指して、X座標を減らす方向（左）に移動することを優先する。左方向への移動が困難な場合、Y座標をゴールに近づける方向に移動する。ゴールは(1,7)なので、Y座標を増やす（下に進む）ことを優先する。不明な地点は周辺地図で確認し、必要に応じて探索する。'}`


計画と方針が更新されました。下に行く

> Finished chain.
エージェントの応答: 下に行く
行動結果: 下に進んだ。


False

思い切って回してみよう。

In [ ]:
while not play.step():
    pass



----------


USER_INPUT: 
(3手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□■■
■■□Ｓ■
■■▼■■
■■□■■
△□□■■


プレイヤーの現在座標: (7, 5)

持ち物: 剣を持っていない

現在の方針: 「ゴール(1,7)を目指して、X座標を減らす方向（左）に移動することを優先する。左方向への移動が困難な場合、Y座標をゴールに近づける方向に移動する。ゴールは(1,7)なので、Y座標を増やす（下に進む）ことを優先する。不明な地点は周辺地図で確認し、必要に応じて探索する。」

前回の行動: 「下に行く」

前回の行動結果: 「下に進んだ。」

行動を「上に行く」「下に行く」の中から指示してください。



> Entering new AgentExecutor chain...
下に行く

> Finished chain.
エージェントの応答: 下に行く
行動結果: 下に進んだ。


----------


USER_INPUT: 
(4手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□Ｓ■
■■□■■
■■▼■■
△□□■■
■■■■■


プレイヤーの現在座標: (7, 6)

持ち物: 剣を持っていない

現在の方針: 「ゴール(1,7)を目指して、X座標を減らす方向（左）に移動することを優先する。左方向への移動が困難な場合、Y座標をゴールに近づける方向に移動する。ゴールは(1,7)なので、Y座標を増やす（下に進む）ことを優先する。不明な地点は周辺地図で確認し、必要に応じて探索する。」

前回の行動: 「下に行く」

前回の行動結果: 「下に進んだ。」

行動を「上に行く」「下に行く」の中から指示してください。



> Entering new AgentExecutor chain...
下に行く

> Finished chain.
エージェントの応答: 下に行く
行動結果: 下に進んだ。


----------


USER_INPUT: 
(5手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■□■■
■■□■■
△□▼■■
■■■■■
■■■■■


プレイヤーの現在座標: (7, 7)

持ち物: 剣を持っていない

現在の方針: 「ゴール

`ConversationSummaryBufferMemory()` がちゃんと動いていたか確かめよう。

In [ ]:
play.agent.memory.chat_memory

InMemoryChatMessageHistory(messages=[AIMessage(content='上に行く', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n(32手目)\n\nまだゴールしていません。\n\nプレイヤーの周辺地図:\n\n◎□□■■\n■■□■■\n■■▼■■\n■■□Ｓ■\n■■□■■\n\n\nプレイヤーの現在座標: (7, 3)\n\n持ち物: 剣\n\n現在の方針: 「ゴール(1,7)を目指す。現在の経路(Y座標が7の経路)ではこれ以上X座標を減らす方向（左）に進めないことが判明したため、Y座標を減らす方向（上）に移動し、X座標が減る新たな経路を探す。具体的には、現在の位置(6,7)から(6,6)、(6,5)、(6,4)と進み、そこからX座標を減らしてゴールを目指す。」\n\n前回の行動: 「上に行く」\n\n前回の行動結果: 「上に進んだ。」\n\n行動を「上に行く」「下に行く」の中から指示してください。\n', additional_kwargs={}, response_metadata={}), AIMessage(content='上に行く', additional_kwargs={}, response_metadata={}), HumanMessage(content='\n(33手目)\n\nまだゴールしていません。\n\nプレイヤーの周辺地図:\n\n■■■■■\n◎□□■■\n■■▼■■\n■■□■■\n■■□Ｓ■\n\n\nプレイヤーの現在座標: (7, 2)\n\n持ち物: 剣\n\n現在の方針: 「ゴール(1,7)を目指す。現在の経路(Y座標が7の経路)ではこれ以上X座標を減らす方向（左）に進めないことが判明したため、Y座標を減らす方向（上）に移動し、X座標が減る新たな経路を探す。具体的には、現在の位置(6,7)から(6,6)、(6,5)、(6,4)と進み、そこからX座標を減らしてゴールを目指す。」\n\n前回の行動: 「上に行く」\n\n前回の行動結果: 「上に進んだ。」\n\n行動を「上に行く」「下に行く」の中から指示してください。\n', additional_kwargs={}, response_m

要約もちゃんと生成されていただろうか？

In [ ]:
play.agent.memory.moving_summary_buffer

'AIは「上に行く」を選択した。これを受け、人間は15手目の状況として、ゴール未到達であること、プレイヤーの現在座標が(7,6)になったこと（周辺地図と共に提示）、持ち物（剣）、AIの方針がY座標が7の経路が行き止まりと判断し、Y座標を減らす方向（上）に移動してX座標が減る新たな経路を探すことに更新され、具体的な移動計画も示されたこと、前回の行動が「上に行く」で結果として「上に進んだ」ことを提示し、AIに「上に行く」「下に行く」の中から次の行動を選択するよう求められた。これに対しAIは再び「上に行く」を選択し、人間は16手目の状況として、引き続きゴール未到達であること、プレイヤーの現在座標が(7,5)になったこと（周辺地図と共に提示）、持ち物（剣）、AIの方針は変わらずY座標を減らす方向（上）に移動しX座標が減る新たな経路を探すものであること、前回の行動が「上に行く」で結果として「上に進んだ」ことを提示し、AIに再度「上に行く」「下に行く」の中から次の行動を選択するよう求めた。これに対しAIは三度「上に行く」を選択し、人間は17手目の状況として、引き続きゴール未到達であること、プレイヤーの現在座標が(7,4)になったこと（周辺地図と共に提示）、持ち物（剣）、AIの方針は変わらずY座標を減らす方向（上）に移動しX座標が減る新たな経路を探すものであること、前回の行動が「上に行く」で結果として「上に進んだ」ことを提示し、AIに「上に行く」「下に行く」「右に行く」の中から次の行動を選択するよう求めた。これに対しAIは「右に行く」を選択し、人間は18手目の状況として、引き続きゴール未到達であること、プレイヤーの現在座標が(8,4)になったこと（周辺地図と共に提示）、持ち物（剣）、AIの方針は変わらずY座標を減らす方向（上）に移動しX座標が減る新たな経路を探すものであること、前回の行動が「右に行く」で結果として「右に進んだ」ことを提示し、AIに「左に行く」の中から次の行動を選択するよう求めた。これに対しAIは「左に行く」を選択し、人間は19手目の状況として、引き続きゴール未到達であること、プレイヤーの現在座標が(7,4)になったこと（周辺地図と共に提示）、持ち物（剣）、AIの方針がX座標を減らす方向（左）に移動しゴールに近づく経路を探索するように更新されたこと、前回の行動が「左

見事ゴールしました！ おそらく再現性がないので、醜い形ですが、再実行はせずに公開してしまいます。